In [6]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=100, metavar='N',
                    help='number of epochs to train (default: 100)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'images_MNIST_VAE/reconstruction_.png' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

if __name__ == "__main__":
    for epoch in range(0, args.epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'images_MNIST_VAE/sample_/' + str(epoch) + '.png')

Train Epoch: 0 [0/60000 (0%)]	Loss: 551.418213
Train Epoch: 0 [640/60000 (1%)]	Loss: 299.302765
Train Epoch: 0 [1280/60000 (2%)]	Loss: 241.228058
Train Epoch: 0 [1920/60000 (3%)]	Loss: 218.135025
Train Epoch: 0 [2560/60000 (4%)]	Loss: 222.720200
Train Epoch: 0 [3200/60000 (5%)]	Loss: 206.495941
Train Epoch: 0 [3840/60000 (6%)]	Loss: 206.609665
Train Epoch: 0 [4480/60000 (7%)]	Loss: 199.939453
Train Epoch: 0 [5120/60000 (9%)]	Loss: 181.539383
Train Epoch: 0 [5760/60000 (10%)]	Loss: 194.648636
Train Epoch: 0 [6400/60000 (11%)]	Loss: 185.033188
Train Epoch: 0 [7040/60000 (12%)]	Loss: 170.297607
Train Epoch: 0 [7680/60000 (13%)]	Loss: 171.630127
Train Epoch: 0 [8320/60000 (14%)]	Loss: 174.890625
Train Epoch: 0 [8960/60000 (15%)]	Loss: 167.212677
Train Epoch: 0 [9600/60000 (16%)]	Loss: 167.392471
Train Epoch: 0 [10240/60000 (17%)]	Loss: 163.593811
Train Epoch: 0 [10880/60000 (18%)]	Loss: 161.639923
Train Epoch: 0 [11520/60000 (19%)]	Loss: 159.135315
Train Epoch: 0 [12160/60000 (20%)]	Loss: 

Train Epoch: 1 [41600/60000 (69%)]	Loss: 115.055573
Train Epoch: 1 [42240/60000 (70%)]	Loss: 110.467667
Train Epoch: 1 [42880/60000 (71%)]	Loss: 114.123978
Train Epoch: 1 [43520/60000 (72%)]	Loss: 115.245178
Train Epoch: 1 [44160/60000 (74%)]	Loss: 112.538986
Train Epoch: 1 [44800/60000 (75%)]	Loss: 111.751862
Train Epoch: 1 [45440/60000 (76%)]	Loss: 115.717773
Train Epoch: 1 [46080/60000 (77%)]	Loss: 115.867249
Train Epoch: 1 [46720/60000 (78%)]	Loss: 113.119331
Train Epoch: 1 [47360/60000 (79%)]	Loss: 115.729301
Train Epoch: 1 [48000/60000 (80%)]	Loss: 113.508400
Train Epoch: 1 [48640/60000 (81%)]	Loss: 108.623154
Train Epoch: 1 [49280/60000 (82%)]	Loss: 110.104561
Train Epoch: 1 [49920/60000 (83%)]	Loss: 113.746361
Train Epoch: 1 [50560/60000 (84%)]	Loss: 109.632561
Train Epoch: 1 [51200/60000 (85%)]	Loss: 117.852455
Train Epoch: 1 [51840/60000 (86%)]	Loss: 112.273369
Train Epoch: 1 [52480/60000 (87%)]	Loss: 112.006851
Train Epoch: 1 [53120/60000 (88%)]	Loss: 114.861023
Train Epoch:

Train Epoch: 3 [23040/60000 (38%)]	Loss: 103.449638
Train Epoch: 3 [23680/60000 (39%)]	Loss: 111.493629
Train Epoch: 3 [24320/60000 (41%)]	Loss: 112.667442
Train Epoch: 3 [24960/60000 (42%)]	Loss: 112.830078
Train Epoch: 3 [25600/60000 (43%)]	Loss: 110.276978
Train Epoch: 3 [26240/60000 (44%)]	Loss: 109.994896
Train Epoch: 3 [26880/60000 (45%)]	Loss: 107.411171
Train Epoch: 3 [27520/60000 (46%)]	Loss: 113.847290
Train Epoch: 3 [28160/60000 (47%)]	Loss: 107.116669
Train Epoch: 3 [28800/60000 (48%)]	Loss: 107.408760
Train Epoch: 3 [29440/60000 (49%)]	Loss: 110.056679
Train Epoch: 3 [30080/60000 (50%)]	Loss: 107.508575
Train Epoch: 3 [30720/60000 (51%)]	Loss: 113.826431
Train Epoch: 3 [31360/60000 (52%)]	Loss: 107.034012
Train Epoch: 3 [32000/60000 (53%)]	Loss: 110.792061
Train Epoch: 3 [32640/60000 (54%)]	Loss: 114.147301
Train Epoch: 3 [33280/60000 (55%)]	Loss: 112.115402
Train Epoch: 3 [33920/60000 (57%)]	Loss: 108.465889
Train Epoch: 3 [34560/60000 (58%)]	Loss: 104.464386
Train Epoch:

Train Epoch: 5 [2560/60000 (4%)]	Loss: 108.542488
Train Epoch: 5 [3200/60000 (5%)]	Loss: 105.007881
Train Epoch: 5 [3840/60000 (6%)]	Loss: 103.366257
Train Epoch: 5 [4480/60000 (7%)]	Loss: 105.573029
Train Epoch: 5 [5120/60000 (9%)]	Loss: 105.230148
Train Epoch: 5 [5760/60000 (10%)]	Loss: 110.369743
Train Epoch: 5 [6400/60000 (11%)]	Loss: 109.799988
Train Epoch: 5 [7040/60000 (12%)]	Loss: 105.803604
Train Epoch: 5 [7680/60000 (13%)]	Loss: 109.496475
Train Epoch: 5 [8320/60000 (14%)]	Loss: 108.123703
Train Epoch: 5 [8960/60000 (15%)]	Loss: 110.599213
Train Epoch: 5 [9600/60000 (16%)]	Loss: 106.657013
Train Epoch: 5 [10240/60000 (17%)]	Loss: 106.362717
Train Epoch: 5 [10880/60000 (18%)]	Loss: 107.511200
Train Epoch: 5 [11520/60000 (19%)]	Loss: 104.725868
Train Epoch: 5 [12160/60000 (20%)]	Loss: 106.900726
Train Epoch: 5 [12800/60000 (21%)]	Loss: 106.435600
Train Epoch: 5 [13440/60000 (22%)]	Loss: 106.822716
Train Epoch: 5 [14080/60000 (23%)]	Loss: 99.212479
Train Epoch: 5 [14720/60000 (2

Train Epoch: 6 [45440/60000 (76%)]	Loss: 107.564377
Train Epoch: 6 [46080/60000 (77%)]	Loss: 105.135750
Train Epoch: 6 [46720/60000 (78%)]	Loss: 105.246628
Train Epoch: 6 [47360/60000 (79%)]	Loss: 109.350449
Train Epoch: 6 [48000/60000 (80%)]	Loss: 101.598175
Train Epoch: 6 [48640/60000 (81%)]	Loss: 107.435562
Train Epoch: 6 [49280/60000 (82%)]	Loss: 109.096405
Train Epoch: 6 [49920/60000 (83%)]	Loss: 114.772209
Train Epoch: 6 [50560/60000 (84%)]	Loss: 110.863892
Train Epoch: 6 [51200/60000 (85%)]	Loss: 110.531265
Train Epoch: 6 [51840/60000 (86%)]	Loss: 106.775360
Train Epoch: 6 [52480/60000 (87%)]	Loss: 105.239731
Train Epoch: 6 [53120/60000 (88%)]	Loss: 107.902245
Train Epoch: 6 [53760/60000 (90%)]	Loss: 104.550690
Train Epoch: 6 [54400/60000 (91%)]	Loss: 107.307182
Train Epoch: 6 [55040/60000 (92%)]	Loss: 108.034096
Train Epoch: 6 [55680/60000 (93%)]	Loss: 103.141373
Train Epoch: 6 [56320/60000 (94%)]	Loss: 102.518860
Train Epoch: 6 [56960/60000 (95%)]	Loss: 104.707047
Train Epoch:

...

Train Epoch: 93 [10240/60000 (17%)]	Loss: 98.995956
Train Epoch: 93 [10880/60000 (18%)]	Loss: 98.935074
Train Epoch: 93 [11520/60000 (19%)]	Loss: 99.642059
Train Epoch: 93 [12160/60000 (20%)]	Loss: 96.941360
Train Epoch: 93 [12800/60000 (21%)]	Loss: 96.413101
Train Epoch: 93 [13440/60000 (22%)]	Loss: 101.399033
Train Epoch: 93 [14080/60000 (23%)]	Loss: 103.252335
Train Epoch: 93 [14720/60000 (25%)]	Loss: 104.193756
Train Epoch: 93 [15360/60000 (26%)]	Loss: 102.009682
Train Epoch: 93 [16000/60000 (27%)]	Loss: 97.773026
Train Epoch: 93 [16640/60000 (28%)]	Loss: 102.589050
Train Epoch: 93 [17280/60000 (29%)]	Loss: 97.987892
Train Epoch: 93 [17920/60000 (30%)]	Loss: 101.290497
Train Epoch: 93 [18560/60000 (31%)]	Loss: 102.476486
Train Epoch: 93 [19200/60000 (32%)]	Loss: 98.316162
Train Epoch: 93 [19840/60000 (33%)]	Loss: 101.836105
Train Epoch: 93 [20480/60000 (34%)]	Loss: 102.586197
Train Epoch: 93 [21120/60000 (35%)]	Loss: 95.978119
Train Epoch: 93 [21760/60000 (36%)]	Loss: 101.181488
Tr

Train Epoch: 94 [50560/60000 (84%)]	Loss: 104.956894
Train Epoch: 94 [51200/60000 (85%)]	Loss: 98.793625
Train Epoch: 94 [51840/60000 (86%)]	Loss: 101.215164
Train Epoch: 94 [52480/60000 (87%)]	Loss: 101.060341
Train Epoch: 94 [53120/60000 (88%)]	Loss: 97.054886
Train Epoch: 94 [53760/60000 (90%)]	Loss: 105.114403
Train Epoch: 94 [54400/60000 (91%)]	Loss: 98.602150
Train Epoch: 94 [55040/60000 (92%)]	Loss: 100.237427
Train Epoch: 94 [55680/60000 (93%)]	Loss: 97.125885
Train Epoch: 94 [56320/60000 (94%)]	Loss: 97.285065
Train Epoch: 94 [56960/60000 (95%)]	Loss: 102.766159
Train Epoch: 94 [57600/60000 (96%)]	Loss: 98.192825
Train Epoch: 94 [58240/60000 (97%)]	Loss: 102.634003
Train Epoch: 94 [58880/60000 (98%)]	Loss: 92.805450
Train Epoch: 94 [59520/60000 (99%)]	Loss: 103.570190
====> Epoch: 94 Average loss: 100.5876
====> Test set loss: 101.2837
Train Epoch: 95 [0/60000 (0%)]	Loss: 98.363464
Train Epoch: 95 [640/60000 (1%)]	Loss: 101.622025
Train Epoch: 95 [1280/60000 (2%)]	Loss: 103.77

Train Epoch: 96 [30720/60000 (51%)]	Loss: 104.536285
Train Epoch: 96 [31360/60000 (52%)]	Loss: 98.495560
Train Epoch: 96 [32000/60000 (53%)]	Loss: 102.490250
Train Epoch: 96 [32640/60000 (54%)]	Loss: 99.925064
Train Epoch: 96 [33280/60000 (55%)]	Loss: 93.149612
Train Epoch: 96 [33920/60000 (57%)]	Loss: 99.690903
Train Epoch: 96 [34560/60000 (58%)]	Loss: 98.775360
Train Epoch: 96 [35200/60000 (59%)]	Loss: 104.747406
Train Epoch: 96 [35840/60000 (60%)]	Loss: 102.802559
Train Epoch: 96 [36480/60000 (61%)]	Loss: 94.013245
Train Epoch: 96 [37120/60000 (62%)]	Loss: 98.191399
Train Epoch: 96 [37760/60000 (63%)]	Loss: 99.901230
Train Epoch: 96 [38400/60000 (64%)]	Loss: 99.545776
Train Epoch: 96 [39040/60000 (65%)]	Loss: 101.288971
Train Epoch: 96 [39680/60000 (66%)]	Loss: 98.696487
Train Epoch: 96 [40320/60000 (67%)]	Loss: 102.434677
Train Epoch: 96 [40960/60000 (68%)]	Loss: 101.221680
Train Epoch: 96 [41600/60000 (69%)]	Loss: 106.297279
Train Epoch: 96 [42240/60000 (70%)]	Loss: 101.245529
Tra

Train Epoch: 98 [9600/60000 (16%)]	Loss: 106.391151
Train Epoch: 98 [10240/60000 (17%)]	Loss: 101.274231
Train Epoch: 98 [10880/60000 (18%)]	Loss: 97.242142
Train Epoch: 98 [11520/60000 (19%)]	Loss: 102.903305
Train Epoch: 98 [12160/60000 (20%)]	Loss: 90.062515
Train Epoch: 98 [12800/60000 (21%)]	Loss: 96.737564
Train Epoch: 98 [13440/60000 (22%)]	Loss: 104.910767
Train Epoch: 98 [14080/60000 (23%)]	Loss: 97.811508
Train Epoch: 98 [14720/60000 (25%)]	Loss: 99.487305
Train Epoch: 98 [15360/60000 (26%)]	Loss: 92.640259
Train Epoch: 98 [16000/60000 (27%)]	Loss: 91.939407
Train Epoch: 98 [16640/60000 (28%)]	Loss: 103.167191
Train Epoch: 98 [17280/60000 (29%)]	Loss: 93.148178
Train Epoch: 98 [17920/60000 (30%)]	Loss: 98.008514
Train Epoch: 98 [18560/60000 (31%)]	Loss: 102.521461
Train Epoch: 98 [19200/60000 (32%)]	Loss: 93.747726
Train Epoch: 98 [19840/60000 (33%)]	Loss: 97.199310
Train Epoch: 98 [20480/60000 (34%)]	Loss: 104.179855
Train Epoch: 98 [21120/60000 (35%)]	Loss: 99.710121
Train 

Train Epoch: 99 [49920/60000 (83%)]	Loss: 98.041359
Train Epoch: 99 [50560/60000 (84%)]	Loss: 101.372604
Train Epoch: 99 [51200/60000 (85%)]	Loss: 101.567871
Train Epoch: 99 [51840/60000 (86%)]	Loss: 106.079170
Train Epoch: 99 [52480/60000 (87%)]	Loss: 102.379814
Train Epoch: 99 [53120/60000 (88%)]	Loss: 100.877487
Train Epoch: 99 [53760/60000 (90%)]	Loss: 98.488853
Train Epoch: 99 [54400/60000 (91%)]	Loss: 96.029877
Train Epoch: 99 [55040/60000 (92%)]	Loss: 99.465736
Train Epoch: 99 [55680/60000 (93%)]	Loss: 104.422455
Train Epoch: 99 [56320/60000 (94%)]	Loss: 105.921425
Train Epoch: 99 [56960/60000 (95%)]	Loss: 97.093079
Train Epoch: 99 [57600/60000 (96%)]	Loss: 94.359474
Train Epoch: 99 [58240/60000 (97%)]	Loss: 98.853317
Train Epoch: 99 [58880/60000 (98%)]	Loss: 106.376839
Train Epoch: 99 [59520/60000 (99%)]	Loss: 101.841797
====> Epoch: 99 Average loss: 100.4499
====> Test set loss: 101.1729
Train Epoch: 100 [0/60000 (0%)]	Loss: 98.343040
Train Epoch: 100 [640/60000 (1%)]	Loss: 97